In [3]:
import sqlite3
import pandas as pd

# Path to your SQLite database
db_path = "../data/data.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# List all tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:")
print(tables)


taxi_data = pd.read_sql_query("SELECT * FROM taxi_data;", conn)

weather_data = pd.read_sql_query("SELECT * FROM weather_data;", conn)


# Close the connection
conn.close()


Tables in the database:
           name
0     taxi_data
1  weather_data


In [7]:
print(weather_data.head)

<bound method NDFrame.head of          name    datetime  tempmax  tempmin  temp  feelslikemax  feelslikemin  \
0    New York  2020-01-01      5.1      1.4   3.3           1.3          -2.5   
1    New York  2020-01-02      9.3      0.6   5.1           7.2          -3.2   
2    New York  2020-01-03      9.3      6.8   8.2           9.2           4.9   
3    New York  2020-01-04     10.2      6.2   8.8          10.2           2.7   
4    New York  2020-01-05      5.4      2.2   3.7           1.5          -2.4   
..        ...         ...      ...      ...   ...           ...           ...   
177  New York  2020-06-26     30.2     21.2  25.9          28.9          21.2   
178  New York  2020-06-27     27.2     22.4  24.9          28.3          22.4   
179  New York  2020-06-28     32.2     22.8  26.5          33.1          22.8   
180  New York  2020-06-29     30.5     21.5  24.9          30.5          21.5   
181  New York  2020-06-30     25.6     20.0  22.5          25.6          20.0  

In [8]:
print(taxi_data.head)

<bound method NDFrame.head of           VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
0              1.0  2020-01-01 00:28:15   2020-01-01 00:33:03   
1              1.0  2020-01-01 00:35:39   2020-01-01 00:43:04   
2              1.0  2020-01-01 00:47:41   2020-01-01 00:53:52   
3              1.0  2020-01-01 00:55:23   2020-01-01 01:00:14   
4              2.0  2020-01-01 00:01:58   2020-01-01 00:04:16   
...            ...                  ...                   ...   
16847773       NaN  2020-06-30 23:05:00   2020-06-30 23:32:00   
16847774       NaN  2020-06-30 23:21:47   2020-06-30 23:25:24   
16847775       NaN  2020-06-30 23:34:00   2020-06-30 23:44:00   
16847776       NaN  2020-06-30 23:22:47   2020-06-30 23:42:01   
16847777       NaN  2020-06-30 23:56:18   2020-07-01 00:27:19   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0                     1.0           1.20         1.0                  N   
1                     1.0           1.2

In [11]:
# Group taxi_data by date and calculate the total passenger count
daily_passenger_count = taxi_data.groupby('tpep_pickup_datetime')['passenger_count'].sum().reset_index()

# Rename the columns for clarity
daily_passenger_count.rename(columns={'passenger_count': 'total_passengers'}, inplace=True)

# Display the aggregated data
print(daily_passenger_count)


        tpep_pickup_datetime  total_passengers
0        2003-01-01 00:07:17               1.0
1        2008-12-31 22:20:23               1.0
2        2008-12-31 23:02:19               6.0
3        2008-12-31 23:02:40               1.0
4        2008-12-31 23:02:48               1.0
...                      ...               ...
6368482  2020-11-01 15:26:33               1.0
6368483  2020-11-01 15:41:04               1.0
6368484  2021-01-02 00:22:00               1.0
6368485  2021-01-02 00:44:08               1.0
6368486  2021-01-02 01:12:10               1.0

[6368487 rows x 2 columns]


In [14]:
# Ensure the datetime column is in datetime format
taxi_data['tpep_pickup_datetime'] = pd.to_datetime(taxi_data['tpep_pickup_datetime'])

# Extract the date from the datetime column
taxi_data['date'] = taxi_data['tpep_pickup_datetime'].dt.date

# Filter data for the date range: 01-2020 to 06-2020 (using datetime.date for comparison)
start_date = pd.to_datetime('2020-01-01').date()
end_date = pd.to_datetime('2020-06-30').date()
filtered_taxi_data = taxi_data[(taxi_data['date'] >= start_date) & 
                               (taxi_data['date'] <= end_date)]

# Group by the extracted date and calculate the total passenger count
daily_passenger_count = filtered_taxi_data.groupby('date')['passenger_count'].sum().reset_index()

# Rename the columns for clarity
daily_passenger_count.rename(columns={'passenger_count': 'total_passengers'}, inplace=True)

# Display the filtered and aggregated data
print(daily_passenger_count.head())


         date  total_passengers
0  2020-01-01          281825.0
1  2020-01-02          253419.0
2  2020-01-03          285181.0
3  2020-01-04          295810.0
4  2020-01-05          258618.0
